### Counting the total number of all errors including undesirable Typos, QnAs and truncated QnAs

In [1]:
!pip install language_tool_python spacy textblob

In [2]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd
import os
import csv
import re
import language_tool_python
import time
import cProfile
from textblob import TextBlob
import spacy
from urllib.parse import urlparse, urlunparse
import locale
import nltk
from nltk.corpus import stopwords
import re



def count_multiple_commas(text):
    if isinstance(text, str):
        pattern = re.compile(r',{2,}')
        return len(re.findall(pattern, text))
    else:
        return text

def replace_multiple_commas(text):
    if isinstance(text, str):
        pattern = re.compile(r',{2,}')
        corrected_text = re.sub(pattern, ', ', text)
        if corrected_text != text:
            print(f"\nOriginal text: {text}")
            print(f"Corrected text: {corrected_text}")
        return corrected_text
    else:
        return text

def comma_cluster_removal_df(df):
    total_comma_count = 0

    for column in df.columns:
        df[column] = df[column].apply(replace_multiple_commas)
        total_comma_count += df[column].apply(count_multiple_commas).sum()

    print(f"Total comma count: {total_comma_count}")
    return df, total_comma_count


def is_undesirable_question_to_count(question):
    if isinstance(question, str):
        count_phrases = [
            
            "what is the title",
            "what is the research topic",
            "what data is used in the study",
            "what data sets are collected in this study",
            "how did the study",
            "what does the arrow in figure 6 represent",
            "what was the publication date of the study",
            "what is the title of the paper",
            "what is the purpose of this study",
            "what was the goal of the study",
            "what data was utilized in the study",
            "what is the source of funding for this study",
            "what was the aim of this study",
            "what are the objectives of the study",
            "what was the main focus of the study",
            "what were the main conclusions of the study",
            "what methods were used in the study",
            "what ratio was used for the analysis",
            "what models are shown in fig 4",
            "what is the conclusion of this study",
            "what do the innovations of this study enable",
            "what is the article about",
            "what is the doi number for the article",
            "where can the tool be accessed",
            "what data has been used",
            "what were the results of the study",
            "what are the key findings of this study",
            "what data sources were used in this study",
            "what are the limitations of this study",
            "where was the research conducted",
            "what are the key words for this article",
            "what is the main objective of this study",
            "what evidence supports the research",
            # Add more phrases here
        ]
        return any(phrase in question.lower() for phrase in count_phrases)
    return False

def is_truncated(sentence):
    
    # Ensure the sentence is converted to a string
    sentence = str(sentence)
    # Define a list of sentence-ending punctuation marks
    sentence_endings = ['.', '!', '?', '."', '!"', '?"', '.”', '!”', '?”']
    
    # Check if the sentence is empty or very short
    if len(sentence) < 2:
        return False  # Not truncated
    
    # Check if the last character of the sentence is a sentence-ending punctuation mark
    if sentence[-1] in sentence_endings:
        return False  # Not truncated
    else:
        print ("\n" + sentence+"\n")
        return True   # Truncated

# Assuming you have defined your functions is_undesirable_question_to_count and is_truncated properly

def count_truncated_questions_and_answers_in_df(df, filtered_data_file):
    #df = pd.read_csv(file_path)
    df.info() 
    columns_with_spaces = df.columns.tolist()
    print(columns_with_spaces) 
    questions = df['Question']
    answers = df['Answer']
        
    questions_count = df['Question'].apply(is_undesirable_question_to_count).sum()

    # Count truncated questions
    truncated_questions_count = df['Question'].apply(is_truncated).sum()

    # Count truncated answers
    truncated_answers_count = df['Answer'].apply(is_truncated).sum()
     
    # Filter out truncated rows
    truncated_questions = []
    truncated_answers = []
    
    # Filter out truncated rows
    not_truncated_indices = []
    for i in range(len(df)):
        if not (is_truncated(questions[i]) or is_truncated(answers[i])):
            not_truncated_indices.append(i)
        else:
            if is_truncated(questions[i]):
                truncated_questions.append(questions[i])
                print(f"Truncated Question {i}: {questions[i]}")
            if is_truncated(answers[i]):
                truncated_answers.append(answers[i])
                print(f"Corresponding Question {i}: {questions[i]}")
                print(f"Truncated Answer  {i}: {answers[i]} \n")
    df = df.iloc[not_truncated_indices]
      
    # Filter out questions and their corresponding answers
    filtered_indices = [i for i, question in enumerate(df['Question']) if is_undesirable_question_to_count(question)]
    filtered_data = pd.DataFrame({
        'Question': df['Question'].iloc[filtered_indices],
        'Answer': df['Answer'].iloc[filtered_indices]
    })
    
    # Print the number of rows in the filtered data
    print("Number of rows in filtered data:", len(filtered_data))
    
    # Create a new DataFrame for remaining data without truncated QnA
    remaining_indices = [i for i in range(len(df)) if i not in filtered_indices]
    remaining_data = df.iloc[remaining_indices]
    
    # Print the number of rows in the remaining data
    print("Number of rows in remaining data:", len(remaining_data))
    
    # Save the filtered data
    filtered_data.to_csv(filtered_data_file, index=False, encoding='utf-8')
    
    # Save the remaining data
    #remaining_data.to_csv(remaining_data_file, index=False, encoding='utf-8')
    
    return questions_count, truncated_questions_count, truncated_answers_count, remaining_data



def save_filtered_data(file_path, filtered_data):
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Question', 'Answer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(filtered_data)

        

def count_errors(text):
    tool = language_tool_python.LanguageTool('en-GB')
    matches = tool.check(text)
    return len(matches)

def LanguageTool_df(df):
    tool = language_tool_python.LanguageTool('en-GB')
    corrected_content = []

    #original_error_count = 0
    #corrected_error_count = 0

    for i in range(len(df)):
        corrected_row = []
        for cell in df.iloc[i]:
            if isinstance(cell, str):
                corrected_cell = tool.correct(cell)
                corrected_cell = corrected_cell.replace(' Answer', 'Answer')
                #original_error_count += count_errors(cell)
                #corrected_error_count += count_errors(corrected_cell)
            else:
                corrected_cell = cell
            corrected_row.append(corrected_cell)
        corrected_content.append(corrected_row)

    LanguageTool_corrected_df = pd.DataFrame(corrected_content, columns=df.columns)

    return LanguageTool_corrected_df #, original_error_count, corrected_error_count

# Assuming you have a function count_errors defined elsewhere in your code

# Example usage:
# corrected_df, original_errors, corrected_errors = LanguageTool_df(your_dataframe)


def validate_and_correct_url_in_dataframe(df):
    def validate_and_correct_url(url):
        if isinstance(url, str):
            try:
                parsed_url = urlparse(url)
                if not parsed_url.scheme:
                    # If the URL doesn't have a scheme (e.g., http://), add "http://"
                    corrected_url = 'http://' + url
                    parsed_url = urlparse(corrected_url)

                if not parsed_url.netloc:
                    # If the URL doesn't have a valid network location, return it as is
                    return url

                # Replace "www" with "www." if it doesn't have a dot after it
                corrected_netloc = re.sub(r'www(?!\.)', 'www.', parsed_url.netloc)

                # Replace "com" with ".com" if it doesn't have a dot before it, and is not followed by a forward slash
                corrected_netloc = re.sub(r'(?<!\.)com(?!/)', '.com', corrected_netloc)
                
                 # Replace "com" with ".org" if it doesn't have a dot before it, and is not followed by a forward slash
                corrected_netloc = re.sub(r'(?<!\.)org(?!/)', '.org', corrected_netloc)
        
        
               # Remove dots from ".com" in the middle of the string
                corrected_netloc = re.sub(r'(?<=\w)\.com(?=\w)', 'com', corrected_netloc)
                
                # Remove dots from ".org" in the middle of the string
                corrected_netloc = re.sub(r'(?<=\w)\.org(?=\w)', 'org', corrected_netloc)
                
                
                 # Replace ".com" at the start of a word with "com"
                corrected_netloc = re.sub(r'\.com(?=\w)', 'com', corrected_netloc)
                
                 # Replace ".com" at the start of a word with "org"
                corrected_netloc = re.sub(r'\.org(?=\w)', 'org', corrected_netloc)
      

                # Reassemble the URL with the corrected netloc
                parsed_url = parsed_url._replace(netloc=corrected_netloc)
                corrected_url = urlunparse(parsed_url)

                return corrected_url

            except ValueError:
                # Invalid URL, return it as is
                return url
        else:
            # If the input is not a string (e.g., it's a float or other non-string type),
            # return it as is
            return url

    if 'Full_Text_URL' in df.columns:
        df['Full_Text_URL'] = df['Full_Text_URL'].apply(validate_and_correct_url)

    return df





def add_space_before_opening_bracket(df):
    #df = pd.read_csv(input_file_path)
    
    for column in df.columns:
        df[column] = df[column].apply(lambda cell: re.sub(r'([A-Za-z])\(', r'\1 (', cell) if isinstance(cell, str) else cell)
    
    #df.to_csv(ammended_space_file, index=False)
    
    return df   # Return the output file path after processing


# Function to correct the truncated starting word of the answers
def correct_truncated_start_word(text: str) -> str:
    if isinstance(text, str):
        # Load the spaCy English model
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)

        # Checking if the start token is capitalized or not
        if doc[0].is_title == False:
            first_token = doc[0].text
            if first_token[0].isdigit() and not first_token[-1].isalnum():
                # Correct the word without modifying the '%' symbol
                corrected_word = ''.join([str(TextBlob(part).correct()) if not part.isalnum() else part for part in first_token.split('%')])
                corrected_text = corrected_word + " " + " ".join([token.text for token in doc[1:]])
                return corrected_text.capitalize()
    
        return text
    else:
        return text

def add_full_stops(df):
# function to add full stops to the end of each “Answer” column.
    df['Answer']=df['Answer'].apply(lambda answer: answer.strip()+  '.' if isinstance(answer,str) else answer)
    return df




# Download NLTK data for part-of-speech tagging
nltk.download('punkt')
nltk.download('stopwords')

# POS tagger
tagger = nltk.pos_tag

# Function to check if a word is a noun or other parts of speech
def is_noun_or_other(word):
    tagged = tagger([word])
    pos_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS', 'WP', 'IN']
    return tagged[0][1] in pos_tags

# Function to identify acronyms that are not stopwords and are nouns or other parts of speech
def identify_and_capitalize_acronyms(df, acronyms_df, persons_df):
    # Preprocess acronyms: remove whitespace and lowercase
    acronyms_df['Acronyms'] = acronyms_df['Acronyms'].str.replace(r'\s+', '').str.lower()
    acronyms = set(acronyms_df['Acronyms'])

    # Get the set of English stopwords
    english_stopwords = set(stopwords.words('english'))

    # Preprocess DataFrame
    df['Question'] = df['Question'].str.lower()
    df['Answer'] = df['Answer'].str.lower()

    # Load names of persons into a set
    persons_set = set(persons_df['name'].str.lower())

    for index, row in df.iterrows():
        q_words = re.findall(r'\b\w+\b', row['Question'])
        a_words = re.findall(r'\b\w+\b', row['Answer'])

        q_matches = [word for word in q_words if word in acronyms and word not in english_stopwords and is_noun_or_other(word) and word not in persons_set]
        a_matches = [word for word in a_words if word in acronyms and word not in english_stopwords and is_noun_or_other(word) and word not in persons_set]

        # Capitalize acronyms
        for word in q_matches:
            row['Question'] = row['Question'].replace(word, word.upper())

        for word in a_matches:
            row['Answer'] = row['Answer'].replace(word, word.upper())

         # Capitalize names from persons_df
        for word in q_words:
            if word in persons_set:
                row['Question'] = row['Question'].replace(word, word.title())

        for word in a_words:
            if word in persons_set:
                row['Answer'] = row['Answer'].replace(word, word.title())

    return df

def punc_removal_at_start(input_df):
    try:
        # Create a copy of the input DataFrame to avoid modifying the original
        df = input_df.copy()

        # Define a function to clean a single string by removing non-alphanumeric characters
        def clean_text(text):
            if isinstance(text, str):
                # Find and remove non-alphanumeric characters from the beginning of the text
                cleaned_text = re.sub(r'^[^a-zA-Z0-9\s]+', '', text)
                return cleaned_text
            else:
                return text  # Return non-string values as is

        # Apply the clean_text function to the 'Question' and 'Answer' columns
        df['Question'] = df['Question'].apply(clean_text)
        df['Answer'] = df['Answer'].apply(clean_text)

        return df

    except Exception as e:
        print("An error occurred:", str(e))
        
# Function to format numbers with commas, preserving non-numeric characters
def format_number_with_commas(number_str):
    try:
        # Check if the input is a numeric value
        if isinstance(number_str, (int, float)):
            return locale.format_string("%d", int(number_str), grouping=True)
        
        # Use regular expression to find and format numbers in the string
        formatted_str = re.sub(r'(\d[\d,]*)', lambda x: locale.format_string("%d", int(x.group(0).replace(',', '')), grouping=True), number_str)
        return formatted_str
    except (ValueError, TypeError):
        return number_str

# Function to format numbers in a DataFrame
def format_numbers_in_dataframe(df):
    df_copy = df.copy()  # Create a copy of the DataFrame to avoid modifying the original
    columns_to_format = ['Question', 'Answer']  # Specify the columns to format
    for column in columns_to_format:
        df_copy[column] = df_copy[column].apply(format_number_with_commas)
    
    return df_copy





 
def main():
    # List of CSV files to process
    start_time = time.time() 
    
    
    
    csv_files = [
       # "C:/Users/HP/Documents/Sam/Data Science Voluteer/CSV/Geothermal_energy_wellcome_20230717.csv",
        #"C:/Users/Joshua Giwa/Downloads/QnAs_generated_27_07_2023/Solar power_wellcome_20230717.csv",
        #"C:/Users/Joshua Giwa/Downloads/QnAs_generated_27_07_2023/Solar power_gatesopen_20230717.csv",
        #"C:/Users/Joshua Giwa/Downloads/QnAs_generated_27_07_2023/Geothermal+energy_f1000_20230717.csv",
       # "C:/Users/Joshua Giwa/Downloads/QnAs_generated_27_07_2023/Geothermal_energy_wellcome_20230717.csv",
      #  "C:/Users/Joshua Giwa/Downloads/QnAs_generated_27_07_2023/Carbon+footprint_wellcome_20230717.csv"
      #  "C:/Users/Joshua Giwa/Downloads/test_dataset.csv"
        #"C:/Users/LolaPwasanga/Downloads/datarecords.csv"
       # 'C:/Users/Joshua Giwa/Downloads/8606-subtopics_Clean.csv'
        'C:/Users/Joshua Giwa/Downloads/QnAs_generated_26_07_2023/QnA_Biomass_f1000_20230717.csv'
        ## Add more file paths here as needed
    ]

    total_questions_count = 0
    total_truncated_questions_count = 0
    total_truncated_answers_count = 0
    

    for csv_file in csv_files: 
        df = pd.read_csv(csv_file)
        #print(df.columns)
        
        #df = add_full_stops(df)

        # Removing any punctuation if any from the start of the answer
        df['Answer'] = df['Answer'].str.replace(r'^[\.\:\- ]*', '', regex=True)     # regex to replace the punctuations from the start of the answer

        # Correcting answers that are truncated in the start of the answer
        df['Answer'] = df['Answer'].apply(correct_truncated_start_word)
        
       # Load AcronymsFile_2.csv into a DataFrame
        acronyms_file = 'C:/Users/Joshua Giwa/Downloads/AcronymsFile_2.csv'
        acronyms_df = pd.read_csv(acronyms_file)

        # Load names_of_persons.csv into a DataFrame
        persons_file = 'C:/Users/Joshua Giwa/Downloads/names_of_persons.csv'
        persons_df = pd.read_csv(persons_file)
        
       # result_df = identify_and_capitalize_acronyms(df, acronyms_df, persons_df)
        
        locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
        
        formatted_df = format_numbers_in_dataframe(df)
        
        
        #df['Full_Text_URL'] = df['Full_Text_URL'].apply(validate_and_correct_url)

        # corrected_file_path = os.path.join(os.path.expanduser("~"), "Downloads", os.path.basename(csv_file).replace('.csv', '_corrected.csv'))
        #remaining_data_file = os.path.join(os.path.expanduser("~"), "Downloads", os.path.basename(csv_file).replace('.csv', '_remaining_data.csv'))
        filtered_data_file = os.path.join(os.path.expanduser("~"), "Downloads", os.path.basename(csv_file).replace('.csv', '_filtered_questions_non_optimized.csv'))
        #ammended_space_file = os.path.join(os.path.expanduser("~"), "Downloads", os.path.basename(csv_file).replace('.csv', 'bracket_spaced.csv'))
        
        punc_removal_at_start_df = punc_removal_at_start(formatted_df)
        questions_count, truncated_questions, truncated_answers, remaining_data = count_truncated_questions_and_answers_in_df(punc_removal_at_start_df, filtered_data_file)
        comma_cluster_removed_df, total_comma_count = comma_cluster_removal_df(remaining_data)
        space_before_bracket_ammended_df = add_space_before_opening_bracket(comma_cluster_removed_df)
        LanguageTool_corrected_df= LanguageTool_df(space_before_bracket_ammended_df)
        validate_and_correct_url_in_df = validate_and_correct_url_in_dataframe(LanguageTool_corrected_df)  
        
       
        
        # Save the processed DataFrame with "updated" added to the name
        cleaned_df =  validate_and_correct_url_in_df.copy()
        cleaned_df_filename = os.path.basename(csv_file).replace('.csv', '_cleaned.csv')

        # Assuming that you have a directory where you want to save the updated DataFrames
        cleaned_df_dir = os.path.join(os.path.expanduser("~"), "Downloads", "cleaned_QnAs_non_optimized")

        if not os.path.exists(cleaned_df_dir):
            os.makedirs(cleaned_df_dir)

        cleaned_df_file_path = os.path.join(cleaned_df_dir, cleaned_df_filename)

        # Save the updated DataFrame as a CSV file
        cleaned_df.to_csv(cleaned_df_file_path, index=False, encoding='utf-8')


        # Now, you can use the updated DataFrame for further processing if needed

        
        
        total_questions_count += questions_count
        total_truncated_questions_count += truncated_questions
        total_truncated_answers_count += truncated_answers
        
        
        print(f"File: {csv_file}\n")
        print(f"Total undesirable questions: {questions_count}")
        print(f"Total Truncated Questions: {truncated_questions}")
        print(f"Total Truncated Answers: {truncated_answers}")
       # print(f"Original typo/error found: {original_error_count}")
        print(f"excessive comma occurence: {total_comma_count}")
        #print(f"Corrected typo/error count: {corrected_error_count}")

        
        print(f"(undesirable_questions + Truncated Questions + Truncated Answers + excessive comma occurence): {questions_count + truncated_questions + truncated_answers +  total_comma_count}\n")

  #  print(f"Total Questions Count: {total_questions_count}")
  #  print(f"Total Truncated Questions Count: {total_truncated_questions_count}")
  #  print(f"Total Truncated Answers Count: {total_truncated_answers_count}")
  #  print(f"Total (Questions + Truncated Questions + Truncated Answers): {total_questions_count + total_truncated_questions_count + total_truncated_answers_count}")

    end_time = time.time()
    
    elapsed_time_seconds = end_time - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60

    print(f"Script ran for {elapsed_time_minutes:.2f} minutes.")
    
    
    
    
if __name__ == "__main__":    
    main()

[nltk_data] Downloading package punkt to C:\Users\Joshua
[nltk_data]     Giwa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Joshua
[nltk_data]     Giwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10219 entries, 0 to 10218
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          10219 non-null  object
 1   DOI            10219 non-null  object
 2   Authors        10219 non-null  object
 3   Full_Text_URL  10219 non-null  object
 4   Paragraph      10219 non-null  object
 5   Question       10213 non-null  object
 6   Answer         10177 non-null  object
dtypes: object(7)
memory usage: 559.0+ KB
['Title', 'DOI', 'Authors', 'Full_Text_URL', 'Paragraph', 'Question', 'Answer']

How did the Pn of white birch respond to high temperature conditions and elevated CO


A):


A):


What parts of Europe have been studied in regards to the temporal dynamics of populations of


How does hydroxymethyl affect the N


How did we determine hydrogen peroxide (H


What is the purpose of the Mans


What was found in the elemental analysis from the surface of Ag coated T


The in vivo maximal carboxylation rate (Vcmax), PAR-saturated electron transport rate (Jmax), triose phosphate utilization (TPU) and other relevant parameters were calculated from the Pn-Ci curve according to Farquhar et al., van Caemmerer and Farquhar, Sharkey, Harley and Sharkey and Harley et al. The Pn-Ci curves were fit using the Photosyn Assistant software (Dundee Scientific, Scotland, UK) to estimate V


The in vivo maximal carboxylation rate (Vcmax), PAR-saturated electron transport rate (Jmax), triose phosphate utilization (TPU) and other relevant parameters were calculated from the Pn-Ci curve according to Farquhar et al., van Caemmerer and Farquhar, Sharkey, Harley and Sharkey and Harley et al. The Pn-Ci curves were fit using the Photosyn Assistant software (Dundee Scientific, Scotland, UK) to estimate V

Corresponding Question 7: What methods and software were used to calculate the relevant parameters from the Pn-Ci curve?
Truncated Answer  7: The in vivo maximal carboxylat


It was necessary to conduct a research to explain the increased survival of h-AMSCs through the treatment of sub-lethal hypoxia precondition (oxygen concentration of 1%) for 24 48 and 72 hours by looking at the expression of inhibition on apoptosis and HSP27 expression, and BCL2. In addition


It was necessary to conduct a research to explain the increased survival of h-AMSCs through the treatment of sub-lethal hypoxia precondition (oxygen concentration of 1%) for 24 48 and 72 hours by looking at the expression of inhibition on apoptosis and HSP27 expression, and BCL2. In addition

Corresponding Question 5527: Why was it necessary to conduct a research to explain the increased survival of h-AMSCs?
Truncated Answer  5527: It was necessary to conduct a research to explain the increased survival of h-AMSCs through the treatment of sub-lethal hypoxia precondition (oxygen concentration of 1%) for 24 48 and 72 hours by looking at the expression of inhibition on apoptosis and HSP27 expressio

Number of rows in filtered data: 23
Number of rows in remaining data: 9771

Original text:  The studies were done by Hall 
                et al.,
                
11
 and Arezzo 
                et al.,
                
9
 stated the VAS did not differ significantly. Meanwhile, Lirici 
                et al.,
                
62
 demonstrated an improvement in VAS after day one, although the pain score on day one did not show any differences between SILC and CMLC. On the other hand, Evers 
                et al.,
                
7
 revealed a superior VAS outcome in the SILC group at 24 hours. The opposite outcomes were reported by Lyu 
                et al.,,
                
12
 in which SILC remained inferior for post-operative pain reduction at 6, 8, 12 and 24 hours. The heterogeneity of VAS was influenced by many elements, including the type of anaesthetic drugs, length of incision, and psychological factors. Thus, our study reported a high degree of heterogeneity. In terms of 

Total comma count: 0
File: C:/Users/Joshua Giwa/Downloads/QnAs_generated_26_07_2023/QnA_Biomass_f1000_20230717.csv

Total undesirable questions: 24
Total Truncated Questions: 46
Total Truncated Answers: 417
excessive comma occurence: 0
(undesirable_questions + Truncated Questions + Truncated Answers + excessive comma occurence): 487

Script ran for 302.02 minutes.


### !jupyter notebook list


In [1]:
from jupyter_core.paths import jupyter_config_dir
jupyter_dir = jupyter_config_dir()
jupyter_dir


'C:\\Users\\Joshua Giwa\\.jupyter'